## Notes

Sea Ice Contration data was obtained from:
/archive/wfc/SPEAR/SPEAR_c192_o1_Hist_AllForc_IC1921_K50_ens_01_03/pp_ens_01/ice/ts/monthly/90yr/ice.192101-201012.CN.nc

- `CN` is **ice concentration per thickness category** -> each slice `CN[:, i, :, :]` gives the fractional sea ice concentration in the `i`-th thickness category at each grid point
- The values are between 0–1 (unitless, fraction of area covered).
- To get the **total sea ice concentration at each grid point** do a simple **sum over the thickness categories**:


In [1]:
import xarray as xr

variable = "CN"
file_name = 'RawData/ice.192101-201012.CN.nc'

data_file = xr.open_mfdataset(file_name, combine='by_coords', decode_timedelta=True)
CN = data_file[variable]

fill_value = CN.attrs.get('_FillValue', None)
if fill_value is not None:
    CN = CN.where(CN != fill_value)  # Mask out fill values before summing

sic = CN.sum(dim="ct", skipna=False)

sic = sic.rename("sic")
sic.attrs["long_name"] = "Sea Ice Concentration"
sic.attrs["units"] = "fraction"
sic.attrs["note"] = "Summed over ice categories (ct)"

# Write the result to a NetCDF file, fill values will remain where they were
sic.to_netcdf("DATA/ice.192101-201012.sic.nc")